## Agenda
1. **Wie funktioniert ein LLM?**  
   - Grundlagen der Transformer-Architektur  
   - Limitierungen von LLMs  

2. **Warum braucht es RAG?**  
   - Statisches Wissen vs. dynamische Daten  
   - Reduktion von Halluzinationen  

3. **Phasen der Dateiverarbeitung für RAG**  
   - Embedding, Snippets, Vektordatenbank  

4. **RAG vs. Finetuning**  
   - Vor- und Nachteile im Vergleich  

## 1. Wie funktioniert ein LLM?

LLMs sind komplexe neuronale Netzwerke, die auf großen Textkorpora trainiert werden.
Sie können eine Vielzahl von Aufgaben ausführen, wie z. B. Textgenerierung, Textklassifizierung und Fragebeantwortung.

LLMs verwenden Selbstaufmerksamkeitsmechanismen (`self-attention`), um die Bedeutung von Wörtern in einem Satz zu verstehen 
und einen kohärenten Text zu generieren. 

Sie sind in der Lage, Kontext und Semantik zu erfassen und so menschliche Sprache zu imitieren.

- **Transformer-Architektur**:  
  - Nutzt **Self-Attention-Mechanismen**, um Kontext in Texten zu erfassen.  
  - Verarbeitet Sequenzen parallel (nicht sequenziell wie RNNs).  

- **Autocomplete auf Steroiden**:  
  - LLMs generieren Text durch Vorhersage des nächsten Tokens basierend auf Trainingsdaten.  
  - Beispiel: Bei der Eingabe *"Erneuerbare Energie..."* ergänzt das Modell *"...ist nachhaltig"*.  

- **Limitierungen**:  
  - **Statisches Wissen**: Trainingsdaten sind "eingefroren" (z. B. GPT-3: Daten bis 2021).  
  - **Kontextfenster**: Begrenzte Token-Kapazität (z. B. 4k–128k Tokens).  


## 2. Warum braucht es RAG?
- **Problem**: LLMs können keine **aktuellen oder domänenspezifischen Daten** abrufen.  
  - Beispiel: *"Wie war das Wetter gestern in Berlin?"* → LLM ohne RAG halluziniert Antworten.  

- **Lösung mit RAG**:  
  - **Retrieval**: Externe Datenbanken (z. B. PDFs, Webseiten) durchsuchen.  
  - **Augmented Generation**: Gefundene Kontexte in den Prompt integrieren.  

- **Vorteile**:  
  - **Aktualität**: Antworten basieren auf Echtzeitdaten.  
  - **Transparenz**: Quellen der Information sind nachvollziehbar.

RAG löst diese Probleme, indem es externe Datenquellen in den Generierungsprozess einbindet. 
Es ermöglicht dem Chatbot, auf aktuelle und relevante Informationen zuzugreifen und 
∏so genauere Antworten zu liefern.

## 3. Phasen der Dateiverarbeitung für RAG

1. **Embedding**: Die zu verarbeitenden Dateien werden in Embedding-Vektoren umgewandelt. Diese Vektoren sind numerische Darstellungen, die die Bedeutung der Dateien erfassen.
2. **Snippets**: Die Dateien werden in kleinere Abschnitte, sogenannte Snippets, zerlegt. Dies erleichtert die Suche und den Abruf relevanter Informationen.
3. **Speicherung in Vektordatenbank**: Die Embedding-Vektoren und Snippets werden in einer Vektordatenbank gespeichert. Diese Datenbank ermöglicht einen effizienten Abruf der Informationen, wenn der Chatbot eine Anfrage erhält.

#### **Embedding-Vektoren**
Embedding-Vektoren sind mathematische Darstellungen, die die Semantik von Texten erfassen. 
Sie werden durch Trainieren eines Embedding-Modells auf großen Textkorpora erstellt. 
Diese Vektoren ermöglichen es, die Ähnlichkeit zwischen Texten zu messen und so relevante Informationen abzurufen.

#### **Snippets**
Snippets sind kleine Textabschnitte, die aus den ursprünglichen Dateien extrahiert werden. 
Sie stellen eine Zusammenfassung der wichtigsten Informationen dar und erleichtern die Suche nach spezifischen Inhalten.

#### **Vektordatenbank**
Eine Vektordatenbank speichert Embedding-Vektoren und ermöglicht es, ähnliche Vektoren effizient abzurufen. 
Sie ist ein entscheidender Bestandteil des RAG-Prozesses, da sie den schnellen Abruf relevanter Informationen ermöglicht.

### Phase 1: **Indexing**  / **Creating Chunks**
1. **Laden**: PDFs, CSVs oder Webseiteninhalte extrahieren (z. B. mit `PyPDFLoader`).  
2. **Teilen**: Dokumente in Chunks zerlegen (z. B. `RecursiveCharacterTextSplitter`).  
3. **Embedding**: Text mit Modellen wie BERT oder OpenAI in Vektoren umwandeln.  
4. **Speichern**: Vektoren in Datenbanken wie FAISS oder Qdrant ablegen.  

![](https://learn.microsoft.com/en-us/azure/architecture/ai-ml/guide/rag/_images/rag-high-level-architecture.svg)

![](https://learn.microsoft.com/en-us/azure/architecture/ai-ml/guide/rag/_images/embedding-similarity.svg)

### Phase 2: **Retrieval & Generierung**  
5. **Ähnlichkeitssuche**: Benutzerfragen werden in Vektoren umgewandelt und mit der Datenbank abgeglichen.  
6. **Antwortgenerierung**: LLM kombiniert Top-Kontexte mit der Frage für präzise Antworten. 

```python
# Beispiel: Text-Splitting mit LangChain
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)
```

## 4. RAG vs. Finetuning
Kriterium | RAG | Finetuning |
--- | --- | --- |
Datenaktualität | Dynamisch (Echtzeit-Datenbanken) | Statisch (Trainingsdatensatz) |
Kosten | Gering (kein erneutes Training) | Hoch (GPU-Ressourcen) |
Anwendungsfall | Breite Domänen + aktuelle Infos | Spezialisierte Tasks (z. B. Medizin) |
Nachteile | Latenz durch Retrieval | Overfitting, Catastrophic Forgetting |


### Empfehlungen:

  - **RAG:** Für Chatbots mit Webseiten/PDF-Anbindung oder FAQs
  - **Finetuning:** Für niche Domänen mit stabilen Daten (z. B. juristische Sprache)

### Zusammenfassung
RAG und Finetuning sind unterschiedliche Ansätze, um die Leistung von LLMs zu verbessern. RAG konzentriert sich auf die Integration externer Datenquellen, während Finetuning das Modell auf eine bestimmte Aufgabe hin optimiert.

  - LLMs sind mächtig, aber durch statisches Wissen begrenzt.
  - RAG ergänzt LLMs mit dynamischen Daten für präzisere Antworten.

## 5. Langchain
Langchain ist ein mächtiges Framework, das die Entwicklung von RAG-basierten Anwendungen vereinfacht und beschleunigt.

**1. Modularität und Flexibilität:** 

Langchain bietet eine modulare Architektur, die es Entwicklern ermöglicht, verschiedene Komponenten des RAG-Prozesses einfach zu integrieren und anzupassen. Sie können verschiedene LLM-Modelle, Embedding-Modelle und Vektordatenbanken auswählen und kombinieren, um die beste Leistung für Ihre spezifischen Anforderungen zu erzielen.

**2. Effiziente Datenverarbeitung:** 

Langchain bietet eine Reihe von Tools und Klassen, die die Verarbeitung und Vorbereitung von Daten für den RAG-Prozess vereinfachen. Es unterstützt die Extraktion von Text aus verschiedenen Quellen, die Tokenisierung und das Erstellen von Embedding-Vektoren. Durch die Nutzung von Langchain können Sie Zeit und Aufwand sparen, da viele dieser Aufgaben bereits implementiert und optimiert sind.

**3. Skalierbarkeit und Leistung:**

Langchain ist darauf ausgelegt, mit großen Datenmengen und komplexen Abfragen umzugehen. Es nutzt effiziente Algorithmen und Datenstrukturen, um den Abruf relevanter Informationen zu beschleunigen. Durch die Einbindung von Langchain können Sie die Leistung Ihres Chatbots verbessern und eine schnellere Antwortzeit erzielen.

**4. Integration mit anderen Tools:**

Langchain bietet eine breite Palette an Integrationen mit anderen beliebten Tools und Bibliotheken. Sie können Langchain mit Streamlit, einem beliebten Framework für die Erstellung interaktiver Web-Anwendungen, kombinieren, um eine benutzerfreundliche Chatbot-Schnittstelle zu erstellen. Langchain unterstützt auch die Integration mit verschiedenen LLM-Anbietern, wie z. B. OpenAI, Cohere und Hugging Face, was Ihnen die Flexibilität gibt, verschiedene Modelle zu nutzen.